In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5

import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
from scipy.optimize import curve_fit


from qick import *
from qick.helpers import gauss

import time
import os
import sys
sys.path.append('/home/xilinx/jupyter_notebooks/')
import scipy as sp
import json
from scipy.fft import fft, fftfreq
import Pyro4.util

from slab.instruments import *
from slab.experiment import Experiment
from slab.datamanagement import SlabFile
from slab import get_next_filename, AttrDict

Could not import QickInstrument (will only work if running on rfsoc). If running on rfsoc, try install/update qick package
Warning failed to import AWG81180A
Warning failed to import Tek5014
Warning failed to import Tek70001
Warning failed to import M8195A


c:\ProgramData\Anaconda3\lib\site-packages\IPython\qt.py:12: ShimWarning: The `IPython.qt` package has been deprecated since IPython 4.0. You should import from qtconsole instead.
  warn("The `IPython.qt` package has been deprecated since IPython 4.0. "


Could not load InstrumentManagerWindow
Could not load labbrick
Could not load Autonics TM4 Driver
Could not load Oxford Trition driver


In [2]:
sys.path.append(os.getcwd()+'/../../qutip_sims')
from QSwitch import QSwitch
from PulseSequence import PulseSequence
import experiments as meas
from TomoAnalysis import TomoAnalysis
tomo_analysis = TomoAnalysis(nb_qubits=3)
import qutip as qt
import matplotlib.style as style
style.use('S:\Connie\prx.mplstyle')

imported experiments.clifford_averager_program
imported experiments.fitting
imported experiments.four_qubit.fourQ_state_tomo
imported experiments.qram_protocol_timestepped
imported experiments.single_qubit.acstarkshift_spectroscopy
imported experiments.single_qubit.amplitude_rabi
imported experiments.single_qubit.length_rabi
imported experiments.single_qubit.pulse_probe_spectroscopy
imported experiments.single_qubit.resonator_spectroscopy
imported experiments.single_qubit.rfsoc_tof_calibration
imported experiments.single_qubit.single_shot
imported experiments.single_qubit.t1
imported experiments.single_qubit.t2_ramsey
imported experiments.three_qubit.threeQ_state_tomo
imported experiments.two_qubit.amplitude_rabi_EgGf
imported experiments.two_qubit.amplitude_rabi_f0g1
imported experiments.two_qubit.amprabi_opt_ctrl_state_prep
imported experiments.two_qubit.crosstalk_echo_calib
imported experiments.two_qubit.length_rabi_EgGf
imported experiments.two_qubit.length_rabi_F0G1
imported exper

In [3]:
psiZ = [qt.basis(2,0), qt.basis(2,1)]
psiX = [1/np.sqrt(2)*(psiZ[0]+psiZ[1]), 1/np.sqrt(2)*(psiZ[0]-psiZ[1])]
psiY = [1/np.sqrt(2)*(psiZ[0]+1j*psiZ[1]), 1/np.sqrt(2)*(psiZ[0]-1j*psiZ[1])]
psi_dict = dict(Z=psiZ, X=psiX, Y=psiY)

psis = dict() # psis in the ZZZ basis
for i, label_numeric in enumerate(tomo_analysis.calib_order_numeric):
    psis.update({label_numeric:tomo_analysis.psi_basis['ZZZ'][i]})
id3q = qt.tensor(qt.qeye(2), qt.qeye(2), qt.qeye(2))
id2q = qt.tensor(qt.qeye(2), qt.qeye(2))

In [4]:
def get_qram_qSLR_state_from_rho(init_rho_SI, post_select=False, post_select_state=None):
    """
    init_rho_SI should be a np array
    """
    # in order |00>, |01>, |10>, |11> for Q0 (switch), Q1 (input)
    SI_to_SLR = [psis['000'], -1j*psis['010'], psis['100'], -1j*psis['101']]
    # SI_to_SLR = [psis['000'], psis['010'], psis['100'], psis['101']]

    assert np.shape(init_rho_SI) == id2q.shape
    rho_final_SLR = 0*qt.ket2dm(psis['000'])
    # print('constructing final state on |switch, out1, out2>')
    for i in range(len(SI_to_SLR)):
        slr_ket_i = SI_to_SLR[i]
        for j in range(len(SI_to_SLR)):
            slr_ket_j = SI_to_SLR[j]
            mat_el = init_rho_SI[i, j]
            rho_final_SLR += mat_el * slr_ket_i * slr_ket_j.dag()
    return rho_final_SLR.unit()

def name_to_state_2q(init_state): # in format |QA>|QB>
    Qa, Qb, _ = init_state.split('>')
    Qa = Qa[1:]
    Qb = Qb[1:]
    psi_name_dict = {
        '0':psiZ[0],
        '1':psiZ[1],
        '0+1':(psiZ[0]+psiZ[1]).unit(),
        '0-1':(psiZ[0]-psiZ[1]).unit(), 
        '0+i':(psiZ[0]+1j*psiZ[1]).unit(), 
        '0-i':(psiZ[0]-1j*psiZ[1]).unit(), 
    }
    try:
        psiA = psi_name_dict[Qa]
    except:
        print(f'QA state {Qa} is not an allowed cardinal state')
    try:
        psiB = psi_name_dict[Qb]
    except:
        print(f'QB state {Qb} is not an allowed cardinal state')
    return qt.tensor(psiA, psiB)


In [5]:
tomo_qubits = [0, 2, 3]

# Save paths

In [6]:
save_plot_path = 'S:\\QRAM\\qram_4QR2\\plots\\paper'

# Formatting

In [7]:
color_switch = '#ff7f50'
color_input = '#468499'
color_q2 = '#E16F8F'
color_basis = '#66CDAA'

def CQ_from_init(init_state):
    _init_state = init_state.replace('|', '')
    q0, q1 = _init_state[:-1].split('>')
    if q0 == '0+1' or q0=='0+i':
        if q1 != '0+1' and q1 !='0+i':
            return 'QsCi'
        return 'QsQi'

    else: 
        if q1 == '0+1' or q1=='0+i':
            return 'CsQi'
        return 'CsCi'

def color_from_cq(cq):
    if cq == 'QsCi':
        return color_switch
    elif cq == 'QsQi':
        return color_q2
    elif cq == 'CsQi':
        return color_input
    elif cq == 'CsCi':
        return color_basis

def latex_from_cq(cq):
    return cq

    if cq == 'QsCi':
        return '$Q_\mathrm{switch}C_\mathrm{input}$'
    elif cq == 'QsQi':
        return '$Q_\mathrm{switch}Q_\mathrm{input}$'
    elif cq == 'CsQi':
        return '$C_\mathrm{switch}Q_\mathrm{input}$'
    elif cq == 'CsCi':
        return '$C_\mathrm{switch}C_\mathrm{input}$'


# Import 2q tomo

In [8]:
# RETRIEVE SAVED 2Q TOMO INITIAL STATES FROM MEASUREMENT
rho_MLE_ZZ_2Q_filepath = 'S:\QRAM\qram_4QR2\data\data_241025\\202412091502_init_rhoMLE_ZZ_2Q_01.npz'

print(f'Using rho_MLE_ZZ for Q0/Q1 from file {rho_MLE_ZZ_2Q_filepath}')

rho_MLE_ZZ_dict = dict()
with np.load(rho_MLE_ZZ_2Q_filepath) as npzfile:
    for key in npzfile.keys():
        rho_MLE_ZZ_dict.update({key:npzfile[key]})
print(f'rho_MLE_ZZ_dict retrieved with init_states {rho_MLE_ZZ_dict.keys()}')

Using rho_MLE_ZZ for Q0/Q1 from file S:\QRAM\qram_4QR2\data\data_241025\202412091502_init_rhoMLE_ZZ_2Q_01.npz
rho_MLE_ZZ_dict retrieved with init_states dict_keys(['|0>|0>', '|0>|1>', '|0>|0+1>', '|0>|0+i>', '|1>|0>', '|1>|1>', '|1>|0+1>', '|1>|0+i>', '|0+1>|0>', '|0+1>|1>', '|0+1>|0+1>', '|0+1>|0+i>', '|0+i>|0>', '|0+i>|1>', '|0+i>|0+1>', '|0+i>|0+i>'])


# Load evolv mats

In [9]:
evol_mats_path = "S:\\QRAM\\qram_4QR2\\evol_mats"

In [10]:
print('Will save evol mats to path', evol_mats_path)
qA, qB, qC = tomo_qubits
evol_mats_filename = f'evol_mats_{qA}{qB}{qC}.npz'

qA, qB, qC = tomo_qubits
evol_mats_file_path = os.path.join(evol_mats_path, evol_mats_filename)
# evol_mats_file_path = os.path.join(evol_mats_path, f'evol_mats_{qA}{qB}{qC}_test.npz')
print(f'Using evol mats from file {evol_mats_file_path}')

evol_mats = dict()
with np.load(evol_mats_file_path) as npzfile:
    for key in npzfile.keys():
        evol_mats.update({key:npzfile[key]})

Will save evol mats to path S:\QRAM\qram_4QR2\evol_mats
Using evol mats from file S:\QRAM\qram_4QR2\evol_mats\evol_mats_023.npz


# Get fids grids and n_tomo

In [11]:
use_init_2q_state = True
plot = True

data_path = "S:\QRAM\qram_4QR2\data\data_241025\\"

fids_grids_filepath_1protocol = "202412081553_ntomocorrected_3Q_023_fids_grids.npz"
fids_grids_filepath_3protocol = "202412082023_ntomocorrected_3Q_023._fids_grids.npz"
fids_grids_filepath_5protocol = "202412090054_ntomocorrected_3Q_023_fids_grids.npz"
fids_grids_filepath_7protocol = "202412131029_ntomocorrected_3Q_023_fids_grids.npz"

fids_grids_filepaths = [fids_grids_filepath_1protocol, fids_grids_filepath_3protocol, fids_grids_filepath_5protocol, fids_grids_filepath_7protocol]

In [12]:
def get_fids_grids(fids_grids_filepath):
    # RETRIEVING MEASURED n_tomo_corrected and fids grids
    print(f"Using fids_grids_dict from file {fids_grids_filepath}")
    fids_grids_dict = dict()

    with np.load(fids_grids_filepath, allow_pickle=True) as npzfile:
        for key in npzfile.keys():
            fids_grids_dict.update({key:npzfile[key]})

    init_states = fids_grids_dict["init_states"]

    if 'saved_files' in init_states: init_states = init_states[:-1]

    print(f'fids_grids_dict retrieved with init_states\n{init_states}')
    all_configs = fids_grids_dict["all_configs"].item() #[0]
    # print(all_configs.keys())

    cfg = AttrDict(all_configs[init_states[0]])
    play_pulses = cfg.expt.play_pulses
    tomo_qubits = cfg.expt.tomo_qubits

    print("play_pulses", play_pulses)

    return fids_grids_dict

In [13]:
fids_grids_dict = get_fids_grids(os.path.join(data_path, fids_grids_filepath_1protocol))

Using fids_grids_dict from file S:\QRAM\qram_4QR2\data\data_241025\202412081553_ntomocorrected_3Q_023_fids_grids.npz
fids_grids_dict retrieved with init_states
['|0>|0>' '|0>|1>' '|0>|0+1>' '|0>|0+i>' '|1>|0>' '|1>|1>' '|1>|0+1>'
 '|1>|0+i>' '|0+1>|0>' '|0+1>|1>' '|0+1>|0+1>' '|0+1>|0+i>' '|0+i>|0>'
 '|0+i>|1>' '|0+i>|0+1>' '|0+i>|0+i>']
play_pulses [2, 1, 4, 3]


Pick best overall phi

In [14]:
def get_optimized_phis(fids_grids_dict, use_init_2q_state, use_ZZ_correction, show_fid_grids=False):
    init_states = fids_grids_dict["init_states"]
    check_init_states = init_states
    
    phis = fids_grids_dict["phis"]

    inner_sweep = phis[2]
    outer_sweep = phis[1]
    y_sweep = outer_sweep
    x_sweep = inner_sweep

    if use_ZZ_correction and use_init_2q_state:
        fids_grids = fids_grids_dict["fids_grids_ZZ_vs_meas"]
    elif use_ZZ_correction and not use_init_2q_state:
        fids_grids = fids_grids_dict["fids_grids_ZZ_vs_ideal"]
    elif not use_ZZ_correction and use_init_2q_state:
        fids_grids = fids_grids_dict["fids_grids_base_vs_meas"]
    elif not use_ZZ_correction and not use_init_2q_state:
        fids_grids = fids_grids_dict["fids_grids_base_vs_ideal"]

    best_avg_fid_avg_opt = 0
    best_fids_avg_opt = []
    best_phis_avg_opt = [0, 0, 0]
    best_phi_indices = [0, 0, 0]
    individual_best_fids = np.zeros((len(init_states)))
    individual_best_phis = np.zeros((len(init_states), 3))
    individual_best_phis_indices = np.zeros((len(init_states), 3), dtype=int)
    for iphi0, phi0 in enumerate(phis[0]):
        for iphi1, phi1 in enumerate(phis[1]):
            for iphi2, phi2 in enumerate(phis[2]):
                avg_fid = 0
                for i_state, init_state in enumerate(init_states):
                    if init_state not in check_init_states: continue
                    fid = fids_grids[i_state][iphi0, iphi1, iphi2]
                    avg_fid += fid
                    if fid > individual_best_fids[i_state]:
                        individual_best_fids[i_state] = fid
                        individual_best_phis[i_state, :] = [phi0, phi1, phi2]
                        individual_best_phis_indices[i_state, :] = [iphi0, iphi1, iphi2]
                avg_fid /= len(check_init_states)
                if avg_fid > best_avg_fid_avg_opt:
                    best_phis_avg_opt = [phi0, phi1, phi2]
                    best_phi_indices = [iphi0, iphi1, iphi2]
                    best_avg_fid_avg_opt = avg_fid

    if show_fid_grids:
        vmax = 1.0
        for i_state in range(len(init_states)):
            # print(init_states[i_state])
            # print(f'best phi Q0 {best_phis_avg_opt[0]}')
            plt.figure()
            # plt.title(f'Fidelities {init_states[i_state]} ($\phi_0: ${best_phis_avg_opt[0]:0.4})')
            plt.xlabel(f'phis Q{tomo_qubits[2]}')
            plt.ylabel(f'phis Q{tomo_qubits[1]}')
            plt.pcolormesh(x_sweep, y_sweep, fids_grids[i_state][best_phi_indices[0], :, :], cmap='viridis', shading='auto')
            plt.colorbar()
            plt.clim(vmin=0, vmax=vmax)
            plt.show()


    for i_state, init_state in enumerate(init_states):
        if init_state not in check_init_states: continue
        best_fids_avg_opt.append(fids_grids[i_state][best_phi_indices[0], best_phi_indices[1], best_phi_indices[2]])

    print('check states', check_init_states)
    print('Best fids with individual rotations', individual_best_fids.tolist())
    print('Avg of best fids with individual rotations', np.average(individual_best_fids))
    print('Phis for best fids with individual rotations', individual_best_phis.tolist())

    # test_indices = individual_best_phis_indices[0]
    # print(test_indices)
    # print(fids_grids[0][test_indices[0], test_indices[1], test_indices[2]], "should be", individual_best_fids[0])
    print('Best avg fid with overall optimal phis', best_avg_fid_avg_opt)
    print('Fids for check states using overall optimal phis', best_fids_avg_opt)
    print('Overall optimal phis', best_phis_avg_opt)
    return individual_best_fids, individual_best_phis, best_avg_fid_avg_opt, best_fids_avg_opt, best_phis_avg_opt

In [15]:
get_optimized_phis(get_fids_grids(os.path.join(data_path, fids_grids_filepath_5protocol)), use_init_2q_state=False, use_ZZ_correction=True, show_fid_grids=True) # ZZ_individual_vs_ideal

Using fids_grids_dict from file S:\QRAM\qram_4QR2\data\data_241025\202412090054_ntomocorrected_3Q_023_fids_grids.npz
fids_grids_dict retrieved with init_states
['|0>|0>' '|0>|1>' '|0>|0+1>' '|0>|0+i>' '|1>|0>' '|1>|1>' '|1>|0+1>'
 '|1>|0+i>' '|0+1>|0>' '|0+1>|1>' '|0+1>|0+1>' '|0+1>|0+i>' '|0+i>|0>'
 '|0+i>|1>' '|0+i>|0+1>' '|0+i>|0+i>']
play_pulses [2, 1, 1, 2, 2, 1, 1, 2, 2, 1, 4, 3]
check states ['|0>|0>' '|0>|1>' '|0>|0+1>' '|0>|0+i>' '|1>|0>' '|1>|1>' '|1>|0+1>'
 '|1>|0+i>' '|0+1>|0>' '|0+1>|1>' '|0+1>|0+1>' '|0+1>|0+i>' '|0+i>|0>'
 '|0+i>|1>' '|0+i>|0+1>' '|0+i>|0+i>']
Best fids with individual rotations [0.9705727957765565, 0.8042358280084659, 0.9010165030318109, 0.9091766689130935, 0.9167999419589756, 0.6530650560121191, 0.8488716454060631, 0.8623007188853337, 0.9180946185316663, 0.7503055713497813, 0.8372031378049452, 0.8384224760868983, 0.9314052864962525, 0.7391120539779328, 0.8492586389426062, 0.8587951712008605]
Avg of best fids with individual rotations 0.8492897570239601

(array([0.9705728 , 0.80423583, 0.9010165 , 0.90917667, 0.91679994,
        0.65306506, 0.84887165, 0.86230072, 0.91809462, 0.75030557,
        0.83720314, 0.83842248, 0.93140529, 0.73911205, 0.84925864,
        0.85879517]),
 array([[ 18.94736842, 189.47368421, 341.05263158],
        [265.26315789, 151.57894737, 113.68421053],
        [132.63157895, 113.68421053,   0.        ],
        [208.42105263, 113.68421053, 189.47368421],
        [151.57894737,  18.94736842, 341.05263158],
        [170.52631579,  56.84210526, 303.15789474],
        [170.52631579,  94.73684211,  94.73684211],
        [  0.        , 151.57894737, 113.68421053],
        [341.05263158, 113.68421053, 170.52631579],
        [303.15789474, 208.42105263, 303.15789474],
        [341.05263158,  56.84210526,  75.78947368],
        [341.05263158,  56.84210526,  94.73684211],
        [322.10526316, 227.36842105, 322.10526316],
        [  0.        , 303.15789474, 341.05263158],
        [341.05263158,  56.84210526,  75.78947

In [16]:
def get_fid_name(use_init_2q_state, use_ZZ_correction, use_avg_opt_phis):
    return 'fidelity_' + ('ZZ' if use_ZZ_correction else 'base') + '_rotated_' + ('individual' if not use_avg_opt_phis else 'opt') + '_vs_' + ('meas' if use_init_2q_state else 'ideal')

def get_rho_qram(fids_grids_dict, rho_qram, use_init_2q_state, use_ZZ_correction, individual_best_phis, best_phis_avg_opt):
    init_states = fids_grids_dict["init_states"]

    z_phi123_avg_opt = tomo_analysis.z_gate_nq(best_phis_avg_opt)

    for use_avg_opt_phis in [True, False]:
        fidelity_name = get_fid_name(use_init_2q_state, use_ZZ_correction, use_avg_opt_phis)
        print(f'Calculating {fidelity_name}')
        for i_state, init_state in enumerate(init_states):
            if use_avg_opt_phis:
                z_phi123 = z_phi123_avg_opt
            else:
                z_phi123 = tomo_analysis.z_gate_nq(individual_best_phis[i_state])

            if use_init_2q_state: init_rho = rho_MLE_ZZ_dict[init_state]
            else: init_rho = qt.ket2dm(name_to_state_2q(init_state)).full()

            rho_id = get_qram_qSLR_state_from_rho(init_rho)

            n_tomo_corrected = fids_grids_dict[init_state]

            rho_MLE = tomo_analysis.get_rho_from_counts(
                n_tomo_raw=n_tomo_corrected,
                n_calib=None,
                correct_readout=False,
                correct_neg_counts=False,
                method='analytical',
                ZZ_correction=use_ZZ_correction,
                evol_mats=evol_mats,
            )

            rho_MLE_rot = (z_phi123*qt.Qobj(rho_MLE, dims=rho_id.dims)*z_phi123.dag()).unit()
            fid_rot = qt.fidelity(rho_MLE_rot, rho_id)**2

            rho_MLE_name = 'rho_' + ('ZZ' if use_ZZ_correction else 'base')
            rho_qram[rho_MLE_name].update({init_state:rho_MLE_rot})
            rho_qram[fidelity_name][init_state] = fid_rot
        print(rho_qram[fidelity_name])
        # rho_qram['avg_'+fidelity_name] = np.average(list(rho_qram[fidelity_name].values()))
    return rho_qram

In [17]:
rho_qrams = []
for fids_grids_filepath in fids_grids_filepaths:
    print()
    rho_qram = {
        'rho_base': {},
        'rho_ZZ': {},

        'fidelity_base_rotated_individual_vs_meas': {},
        'fidelity_base_rotated_individual_vs_ideal': {},
        'fidelity_ZZ_rotated_individual_vs_meas': {},
        'fidelity_ZZ_rotated_individual_vs_ideal': {},
        'fidelity_base_rotated_opt_vs_meas': {},
        'fidelity_base_rotated_opt_vs_ideal': {},
        'fidelity_ZZ_rotated_opt_vs_meas': {},
        'fidelity_ZZ_rotated_opt_vs_ideal': {},
        }
    fids_grids_dict = get_fids_grids(os.path.join(data_path, fids_grids_filepath))
    for use_init_2q_state in [True, False]:
        for use_ZZ_correction in [True, False]:
            individual_best_fids, individual_best_phis, best_avg_fid_avg_opt, best_fids_avg_opt, best_phis_avg_opt = get_optimized_phis(fids_grids_dict, use_init_2q_state, use_ZZ_correction, show_fid_grids=False)
            rho_qram = get_rho_qram(fids_grids_dict, rho_qram, use_init_2q_state, use_ZZ_correction, individual_best_phis, best_phis_avg_opt)
            print()
    rho_qrams.append(rho_qram)
# print(rho_qram)


Using fids_grids_dict from file S:\QRAM\qram_4QR2\data\data_241025\202412081553_ntomocorrected_3Q_023_fids_grids.npz
fids_grids_dict retrieved with init_states
['|0>|0>' '|0>|1>' '|0>|0+1>' '|0>|0+i>' '|1>|0>' '|1>|1>' '|1>|0+1>'
 '|1>|0+i>' '|0+1>|0>' '|0+1>|1>' '|0+1>|0+1>' '|0+1>|0+i>' '|0+i>|0>'
 '|0+i>|1>' '|0+i>|0+1>' '|0+i>|0+i>']
play_pulses [2, 1, 4, 3]
check states ['|0>|0>' '|0>|1>' '|0>|0+1>' '|0>|0+i>' '|1>|0>' '|1>|1>' '|1>|0+1>'
 '|1>|0+i>' '|0+1>|0>' '|0+1>|1>' '|0+1>|0+1>' '|0+1>|0+i>' '|0+i>|0>'
 '|0+i>|1>' '|0+i>|0+1>' '|0+i>|0+i>']
Best fids with individual rotations [0.9874963176411256, 0.9557426274203144, 0.9503790128673132, 0.9557527573386293, 0.9915900901592332, 0.8668184472953496, 0.9491626374569251, 0.9535101309634894, 0.9709556332576208, 0.9873449662788828, 0.9736435607390157, 0.9863090401366302, 0.9693583925052545, 0.9683147908966125, 0.9435540894908803, 0.9444586172901274]
Avg of best fids with individual rotations 0.9596494444835877
Phis for best fids wit

 s:\Connie\experiments\qram_tprocv1_expts\TomoAnalysis.py: 1178ComplexWarning: Casting complex values to real discards the imaginary part
 s:\Connie\experiments\qram_tprocv1_expts\TomoAnalysis.py: 1183

{'|0>|0>': 0.9816065116135533, '|0>|1>': 0.9497309736126357, '|0>|0+1>': 0.8350616460885572, '|0>|0+i>': 0.8186523381024576, '|1>|0>': 0.9861305605400738, '|1>|1>': 0.8500701269892601, '|1>|0+1>': 0.9468564274856147, '|1>|0+i>': 0.9470524085519021, '|0+1>|0>': 0.9647491688744803, '|0+1>|1>': 0.9854086830100176, '|0+1>|0+1>': 0.9480920135755452, '|0+1>|0+i>': 0.9785393147228214, '|0+i>|0>': 0.9555364259682384, '|0+i>|1>': 0.9446986917308693, '|0+i>|0+1>': 0.9284995831301158, '|0+i>|0+i>': 0.9395071409870442}
Calculating fidelity_ZZ_rotated_individual_vs_meas
{'|0>|0>': 0.9874963176411256, '|0>|1>': 0.9557426274203144, '|0>|0+1>': 0.9503790128673132, '|0>|0+i>': 0.9557527573386293, '|1>|0>': 0.9915900901592332, '|1>|1>': 0.8668184472953496, '|1>|0+1>': 0.9491626374569251, '|1>|0+i>': 0.9535101309634894, '|0+1>|0>': 0.9709556332576208, '|0+1>|1>': 0.9873449662788828, '|0+1>|0+1>': 0.9736435607390157, '|0+1>|0+i>': 0.9863090401366302, '|0+i>|0>': 0.9693583925052545, '|0+i>|1>': 0.968314790

# Plotting

## Decay of fidelity over number of protocols

Average over init states, binned into cqs

In [18]:
use_fid = 'fidelity_base_rotated_individual_vs_meas'
check_init_states = None # set to None to average over all states, otherwise will only average over specified states

In [19]:
x_sweep = [0, 1, 2, 3]

CQ_fids_dict_all_files = dict(
    CsCi=[], # average fid for each state in this category from all files
    CsQi=[],
    QsCi=[],
    QsQi=[],
)

for i_file, rho_qram in enumerate(rho_qrams):
    CQ_fids_dict = dict(
        CsCi=[], # fid for each state in this category from this file
        CsQi=[],
        QsCi=[],
        QsQi=[],
    )
    fids = rho_qram[use_fid] # dict: init_state -> fidelity for given number of protocols
    print(fids)
    for i, init_state in enumerate(fids.keys()):
        if check_init_states is not None and init_state not in check_init_states: continue
        cq = CQ_from_init(init_state)
        CQ_fids_dict[cq].append(fids[init_state])
        print(f"from file {i_file} adding fid {fids[init_state]} of {init_state} to {cq}")
    for cq in CQ_fids_dict.keys():
        CQ_fids_dict_all_files[cq].append(np.average(CQ_fids_dict[cq]))

{'|0>|0>': 0.9873403790393478, '|0>|1>': 0.9558235982117242, '|0>|0+1>': 0.9507963235284637, '|0>|0+i>': 0.9566862737662111, '|1>|0>': 0.9911532774243534, '|1>|1>': 0.8653184948103015, '|1>|0+1>': 0.9503077163507032, '|1>|0+i>': 0.9480135043978262, '|0+1>|0>': 0.9704002969424286, '|0+1>|1>': 0.9800030918601427, '|0+1>|0+1>': 0.9619213926943108, '|0+1>|0+i>': 0.9762129824222614, '|0+i>|0>': 0.9713684607448989, '|0+i>|1>': 0.9676738915261921, '|0+i>|0+1>': 0.9664142607274997, '|0+i>|0+i>': 0.9653591393955314}
from file 0 adding fid 0.9873403790393478 of |0>|0> to CsCi
from file 0 adding fid 0.9558235982117242 of |0>|1> to CsCi
from file 0 adding fid 0.9507963235284637 of |0>|0+1> to CsQi
from file 0 adding fid 0.9566862737662111 of |0>|0+i> to CsQi
from file 0 adding fid 0.9911532774243534 of |1>|0> to CsCi
from file 0 adding fid 0.8653184948103015 of |1>|1> to CsCi
from file 0 adding fid 0.9503077163507032 of |1>|0+1> to CsQi
from file 0 adding fid 0.9480135043978262 of |1>|0+i> to CsQi

In [47]:
saveplot = True

# fig, ax = plt.subplots(1, 1, figsize=(5, 3))
fig, ax = plt.subplots(1, 1, figsize=(3.5, 3.5))

for cq in CQ_fids_dict_all_files.keys():
    print(cq)
    fids_per_U = CQ_fids_dict_all_files[cq]
    # err_per_U = np.diff(fids_per_U)
    # avg_err_per_U = np.average(np.abs(err_per_U))/2

    denominator = np.roll(fids_per_U, 1)[1:]
    gate_fids_per_U2 = fids_per_U[1:] / denominator
    gate_fids_per_U = np.sqrt(gate_fids_per_U2)
    err_per_U = 1 - gate_fids_per_U
    avg_err_per_U = np.average(np.abs(err_per_U))
    print("denominator", denominator)

    print("fids_per_U", fids_per_U)
    print("err_per_U", err_per_U)
    print("avg_err_per_U", avg_err_per_U)
    plt.plot(range(len(fids_per_U)), fids_per_U, 'o-', label="$\epsilon /U$="+f"{avg_err_per_U*100:.2}%", color=color_from_cq(cq), markersize=5)

ax.set_xticks(range(len(fids_per_U)))
ymin = 0.80
ymax = 1.0
ytick_spacing = 0.1
ytick_labels = np.round(np.linspace(ymin, ymax, int(np.ceil((ymax-ymin)/ytick_spacing))+1) /ytick_spacing) * ytick_spacing
ax.set_yticks(ytick_labels)
ax.tick_params(axis='both', which='major', labelsize=16)
plt.xlabel('$N$ for $(U_{\mathrm{route}}U_{\mathrm{route}}^{-1})^N U_{\mathrm{route}}$', fontsize=16)
plt.ylabel('Average Fidelity', fontsize=16)
plt.ylim(ymin, ymax)
plt.legend(fontsize=14, loc='lower left', frameon=False)
plt.tight_layout()

fids_grids_filepaths_str = '_'.join([fids_grids_filepath[:12] for fids_grids_filepath in fids_grids_filepaths])
filename = f'protocol_repeat_{use_fid}_{fids_grids_filepaths_str}'
print(filename)
if saveplot:
    savename = os.path.join(save_plot_path, filename+'.svg')
    plt.savefig(savename, bbox_inches='tight')
    print("Saved svg plot", savename)

    savename = os.path.join(save_plot_path, filename+'.pdf')
    plt.savefig(savename, bbox_inches='tight')
    print("Saved pdf plot", savename)

    plt.close()
else:
    plt.show()

CsCi
denominator [0.94990894 0.91864846 0.89137645]
fids_per_U [0.9499089373714318, 0.9186484555386506, 0.8913764485751, 0.879083854284943]
err_per_U [0.01659211 0.01495538 0.00691923]
avg_err_per_U 0.01282223932598714
CsQi
denominator [0.95145095 0.92709067 0.91112047]
fids_per_U [0.951450954510801, 0.9270906701798056, 0.9111204717568429, 0.8674184953945259]
err_per_U [0.01288466 0.00865049 0.02427724]
avg_err_per_U 0.015270794309997079
QsCi
denominator [0.97236144 0.93869472 0.89276728]
fids_per_U [0.9723614352684156, 0.9386947183764582, 0.8927672781672428, 0.8791558707002163]
err_per_U [0.01746433 0.02477024 0.00765244]
avg_err_per_U 0.016629003756945055
QsQi
denominator [0.96747694 0.92811216 0.88825578]
fids_per_U [0.9674769438099008, 0.9281121603995761, 0.8882557848232621, 0.8338248160538383]
err_per_U [0.0205553  0.02170735 0.03112358]
avg_err_per_U 0.02446207772503876
protocol_repeat_fidelity_ZZ_rotated_individual_vs_meas_202412081553_202412082023_202412090054_202412131029
Save

## Error breakdown

In [24]:
x_sweep = [0, 1, 2, 3]

i_file = 0 # index in file list to use the data from

CQ_fids_dict_processing_dict = dict() # dict: use_fid -> CQ -> average fid
for use_init_2q_state in [True, False]:
    for use_ZZ_correction in [True, False]:
        for use_avg_opt_phis in [True, False]:
            use_fid = get_fid_name(use_init_2q_state, use_ZZ_correction, use_avg_opt_phis)

            CQ_fids_dict = dict(
                CsCi=[], # fid for each state in this category from this file
                CsQi=[],
                QsCi=[],
                QsQi=[],
            )

            rho_qram = rho_qrams[i_file]
            fids = rho_qram[use_fid] # dict: init_state -> fidelity for given number of protocols
            # print(fids)
            for i, init_state in enumerate(fids.keys()):
                if check_init_states is not None and init_state not in check_init_states: continue
                cq = CQ_from_init(init_state)
                CQ_fids_dict[cq].append(fids[init_state])
                # print(f"from file {i_file} adding fid {fids[init_state]} of {init_state} to {cq}")
            for cq in CQ_fids_dict.keys():
                CQ_fids_dict[cq] = np.average(CQ_fids_dict[cq])

            CQ_fids_dict_processing_dict[use_fid] = CQ_fids_dict

for key in CQ_fids_dict_processing_dict.keys():
    print(key)
    print(CQ_fids_dict_processing_dict[key])

fidelity_ZZ_rotated_opt_vs_meas
{'CsCi': 0.9418845431888807, 'CsQi': 0.886905705057133, 'QsCi': 0.9625982423959014, 'QsQi': 0.9486595131038816}
fidelity_ZZ_rotated_individual_vs_meas
{'CsCi': 0.9504118706290057, 'CsQi': 0.9522011346565893, 'QsCi': 0.9739934457345927, 'QsQi': 0.9619913269141634}
fidelity_base_rotated_opt_vs_meas
{'CsCi': 0.9409289260568954, 'CsQi': 0.8865376218015002, 'QsCi': 0.9611099127094197, 'QsQi': 0.9563277052409578}
fidelity_base_rotated_individual_vs_meas
{'CsCi': 0.9499089373714318, 'CsQi': 0.951450954510801, 'QsCi': 0.9723614352684156, 'QsQi': 0.9674769438099008}
fidelity_ZZ_rotated_opt_vs_ideal
{'CsCi': 0.9107153344392278, 'CsQi': 0.8584689805025049, 'QsCi': 0.9158761267864731, 'QsQi': 0.9123619584861999}
fidelity_ZZ_rotated_individual_vs_ideal
{'CsCi': 0.9107183367665708, 'CsQi': 0.9137832051051373, 'QsCi': 0.9378224745642513, 'QsQi': 0.9238598858747691}
fidelity_base_rotated_opt_vs_ideal
{'CsCi': 0.9110650459447088, 'CsQi': 0.8618224820417274, 'QsCi': 0.933

In [45]:
saveplot = True

# fig, ax = plt.subplots(1, 1, figsize=(5, 2.5))
fig, ax = plt.subplots(1, 1, figsize=(3.5, 3))
cqs = ['CsCi', 'CsQi', 'QsCi', 'QsQi']
cq_labels = [latex_from_cq(cq) for cq in cqs]

use_ZZ_correction = True

# use_fid = get_fid_name(use_init_2q_state=True, use_ZZ_correction=False, use_avg_opt_phis=True)
# fids = [CQ_fids_dict_processing_dict[use_fid][cq] for cq in cqs]
fids = [1.0]*4
colors_cq = [color_from_cq(cq) for cq in cqs]
ax.bar(cq_labels, fids, color=colors_cq, alpha=1.0, edgecolor=colors_cq, linewidth=2.0, linestyle='-', fill=False, label='Theory')

# use_fid = get_fid_name(use_init_2q_state=True, use_ZZ_correction=False, use_avg_opt_phis=True)
# fids = [CQ_fids_dict_processing_dict[use_fid][cq] for cq in cqs]
# colors_cq = [color_from_cq(cq) for cq in cqs]
# ax.bar(cq_labels, fids, color=colors_cq, alpha=1.0, edgecolor=colors_cq, linewidth=0.75, linestyle='--', fill=False, label='Base, $\\vec{\phi}_0$')

use_fid = get_fid_name(use_init_2q_state=False, use_ZZ_correction=use_ZZ_correction, use_avg_opt_phis=True)
label = '$\\rho_{\mathrm{ideal}}$, $\\vec{\phi}_0$'
fids = [CQ_fids_dict_processing_dict[use_fid][cq] for cq in cqs]
colors_cq = [color_from_cq(cq) for cq in cqs]
ax.bar(cq_labels, fids, color=colors_cq, alpha=1.0, edgecolor=colors_cq, linewidth=2.0, linestyle='--', fill=False, label=label)

use_fid = get_fid_name(use_init_2q_state=True, use_ZZ_correction=use_ZZ_correction, use_avg_opt_phis=True)
label = '$\\rho_{\mathrm{meas}}$, $\\vec{\phi}_0$'
fids = [CQ_fids_dict_processing_dict[use_fid][cq] for cq in cqs]
colors_cq = [color_from_cq(cq) for cq in cqs]
ax.bar(cq_labels, fids, color=colors_cq, alpha=1.0, edgecolor=colors_cq, linewidth=2.0, linestyle=':', fill=False, label=label)

use_fid = get_fid_name(use_init_2q_state=True, use_ZZ_correction=use_ZZ_correction, use_avg_opt_phis=False)
label = '$\\rho_{\mathrm{meas}}$, $\{ \\vec{\phi}_k\}$'
fids = [CQ_fids_dict_processing_dict[use_fid][cq] for cq in cqs]
colors_cq = [color_from_cq(cq) for cq in cqs]
ax.bar(cq_labels, fids, color=colors_cq, alpha=0.5, edgecolor=colors_cq, linewidth=0.0, fill=True, label=label)


ymin = 0.80
ymax = 1.01
ytick_spacing = 0.1
ytick_labels = np.round(np.linspace(ymin, ymax, int(np.ceil((ymax-ymin)/ytick_spacing))+1) /ytick_spacing) * ytick_spacing
ax.set_yticks(ytick_labels)
ax.tick_params(axis='x', which='major', labelsize=16)
ax.tick_params(axis='y', which='major', labelsize=16)
# for tick in ax.get_xticklabels(): # tilt the x labels by 45 degrees
    # tick.set_rotation(45)
plt.ylabel('Average Fidelity', fontsize=16)
plt.ylim(ymin, ymax)
# plt.legend(bbox_to_anchor=(1.0, 0.5), loc='center left', frameon=True, edgecolor='k', fontsize=13) # bbox is where to put box, loc is which corner of box is anchored
plt.legend(bbox_to_anchor=(0.45, -0.1), loc='upper center', frameon=True, edgecolor='k', fontsize=16, ncol=2) # bbox is where to put box, loc is which corner of box is anchored
# plt.tight_layout()

fids_grids_filepaths_str = '_'.join([fids_grids_filepath[:12] for fids_grids_filepath in fids_grids_filepaths])
filename = f'cq_err_breakdown_1protocol_{fids_grids_filepaths_str}'
print(filename)
if saveplot:
    savename = os.path.join(save_plot_path, filename+'.svg')
    plt.savefig(savename, bbox_inches='tight')
    print("Saved svg plot", savename)

    savename = os.path.join(save_plot_path, filename+'.pdf')
    plt.savefig(savename, bbox_inches='tight')
    print("Saved pdf plot", savename)

    plt.close()
else:
    plt.show()

cq_err_breakdown_1protocol_202412081553_202412082023_202412090054_202412131029
Saved svg plot S:\QRAM\qram_4QR2\plots\paper\cq_err_breakdown_1protocol_202412081553_202412082023_202412090054_202412131029.svg
Saved pdf plot S:\QRAM\qram_4QR2\plots\paper\cq_err_breakdown_1protocol_202412081553_202412082023_202412090054_202412131029.pdf
